#**Assignment-1**

#### 1. Write a program that takes an initial webpage url as an input and a number. It then finds all out links of the webpage and traverses the links in breadth first traversal till the shortest distance between a webpage and the initial webpage is within the given number. It outputs website url, heading and parent url in an excel file. There should be no repetition. A webpage should be occurring only once in the output. [10]

In [2]:
# Importing the necessary Libraries
from queue import Queue, Empty
import threading, queue
import sys
import time
import urllib.request 
import urllib.request as ur
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [8]:
# taking the input of initial webpage url and the number
initial_url = str(input("Enter the Initial Webpage Url:"))
depth_limit = int(input("Enter the Number:"))

def crawl():
    
    # Set a crawl depth limit for our BFS algorithm:
        print("-" * 80)
        print("Let's do this!\n\n")
        print("Crawling in progress...\n\n")

        # Applying the breadth-first search (BFS) algorithm except in this case
        # We need to to keep track of the depth level and stop the crawl when the
        # Deepest level is reached. 
        found_urls = []
        q =  queue.Queue()
        q.put(initial_url)
        current_level = 0
        # Begin the BFS Algorithm...
        while not q.empty():
            current_url = q.get()
            if current_url not in found_urls:
                found_urls.append(current_url)
                exData(current_url)
                if current_level != depth_limit:
                    current_level = current_level + 1
                    for link in getLinks(current_url, initial_url):
                        q.put(link)
            # Suspend execution for a second.
            time.sleep(1)
        print("-" * 80)

#Function to getlinks of Parent URL
def getLinks(parent_url, initial_url):
    child_urls = []
    try:
        opener =  ur.urlopen(parent_url)
        opener.addheaders = [("User-agent", "Mozilla/5.0")]
        response = opener
        html = response.read()
        soup = BeautifulSoup(html, "lxml")
        anchor_tag_links = soup("a")
        for link in anchor_tag_links:
            if link.has_attr("href"):
                # Convert url from a relative path to absolute by joining 
                # both parent and found link href address.
                temp_url = urljoin(parent_url, link["href"])
                # If initial_url url is found in the temp_url, then great!
                # If not, then send off the temp_url to the exCheck()
                # function to then display why it was detected.
                if initial_url in temp_url:
                    child_urls.append(temp_url)
                else:
                    exCheck(temp_url)

    # Some HTTP Request Errors may pop up! 
    except urllib.error.HTTPError as e:
        # If Error like 404 is requested, continue onwards...
        pass

    # Else, if not, continue on and return a set version of the list 
    # to get rid of duplicate url(s) crawled already.
    return set(child_urls)

# Any url(s) found to be external, we shall list them and list why
# they have been deemed as external. So let's check them.

def exCheck(external_url):
    # Set up an array for external keywords found in url(s) from website
    ignore = ['support']
    # For each item in the ignore list, if external_url matches
    for item in ignore:
        if item in external_url:
            print("-----> Found External URL using keyword: " + item + ":", external_url)


# A function to extract data from every url that comes up in our BFS crawl
# algorithm. Using BS4
def exData(url):
    print("|-----> Outlinks of the webpage is: ", url)
    try:
        opener =  ur.urlopen(url)
        opener.addheaders = [('User-agent', 'Mozilla/5.0')]
        response = opener
        html = response.read()
        # Now parse any page title tags...
        soup = BeautifulSoup(html, "lxml")
        title = soup.find_all('title')
        # Send any page titles to the printMatch() with the parent url...
        printMatch(title, url, 'Heading of the Page is:')
        url_parent = ("--> The Parent url is: " + initial_url)
        print(url_parent)
        print()

    # If HTTP Request error here, then continue onwards again...
    except urllib.error.HTTPError as e:
        pass


def printMatch(title, url, type):
    # for title_item in title:
    title_matches = title

    # For every page visited, for matches found via regex above, print 
    # page titles and the url it has found the match.
    matching_data = []
    for title in title_matches:
        if title not in matching_data:
            # Add both url and title to matching_data
            matching_data.extend(title)
            if type == "Heading is:":
                print(" ->", type, title[0])
            else:
                print(" ->", type, title.get_text())

  
# Calling the Crawler
data = crawl()
dataframe = pd.DataFrame(data)



Enter the Initial Webpage Url:https://www.w3schools.com/
Enter the Number:2
--------------------------------------------------------------------------------
Let's do this!


Crawling in progress...


|-----> Outlinks of the webpage is:  https://www.w3schools.com/
 -> Heading of the Page is: W3Schools Online Web Tutorials
--> The Parent url is: https://www.w3schools.com/

|-----> Outlinks of the webpage is:  https://www.w3schools.com/js/js_json.asp
 -> Heading of the Page is: JavaScript JSON
--> The Parent url is: https://www.w3schools.com/

|-----> Outlinks of the webpage is:  https://www.w3schools.com/js/js_json_intro.asp
 -> Heading of the Page is: JSON Introduction
--> The Parent url is: https://www.w3schools.com/

|-----> Outlinks of the webpage is:  https://www.w3schools.com/asp/asp_ref_response.asp
 -> Heading of the Page is: ASP Response Object
--> The Parent url is: https://www.w3schools.com/

|-----> Outlinks of the webpage is:  https://www.w3schools.com/css/tryit.asp?filename

In [ ]:
# Convert DataFrame into excel file
dataframe.to_excel('Results.xlsx', index = False)